In [1]:
import os
import time
import nd2
import tifffile
from warnings import filterwarnings
from tqdm import tqdm
# Suppress specific warnings if needed
filterwarnings("ignore", category=UserWarning)

In [2]:
# Set input and output directories
data_directory = r"L:\43-RVZ\AIMicroscopy\Mitarbeiter\2_Data\1_NikonTi2\FK_P001_EX004_2025_03_04 60xWI 1.5x KEIO Plate61\AF"
output_directory = data_directory

In [3]:
# Function to convert ND2 to TIF
def convert_nd2_to_tif(nd2_file_path, output_dir, progress_bar):
    try:
        with nd2.ND2File(nd2_file_path) as dataset:
            img_data = dataset.asarray()
            num_z = img_data.shape[0]
            
            for z in range(num_z):
                z_slice = img_data[z]
                output_filename = os.path.basename(nd2_file_path).replace(
                    '.nd2', f'_Z{z+1}.tif'
                )
                output_path = os.path.join(output_dir, output_filename)
                tifffile.imwrite(output_path, z_slice)
                
                # Update progress for each Z-slice
                progress_bar.update(1)
                progress_bar.set_postfix({
                    'File': os.path.basename(nd2_file_path)[:15] + "...",
                    'Elapsed': f"{progress_bar.format_dict['elapsed']:.1f}s"
                })
                
    except Exception as e:
        progress_bar.write(f"Error processing {nd2_file_path}: {str(e)}")

In [4]:
# Batch process all ND2 files in the directory
def batch_convert_nd2_to_tif(data_directory, output_directory):
    # Get list of ND2 files
    nd2_files = [
        os.path.join(data_directory, f) 
        for f in os.listdir(data_directory) 
        if f.lower().endswith('.nd2')
    ]
    
    # Check if files are found
    if not nd2_files:
        print("No .nd2 files found in the specified directory.")
        return  # Exit the function if no files are found
    
    # Print the number of files found
    print(f"Number of .nd2 files found: {len(nd2_files)}")
    
    # Pre-scan to get total Z-slices
    total_z = 0
    with tqdm(nd2_files, desc="Scanning files", unit="file") as scan_bar:
        for f in scan_bar:
            try:
                with nd2.ND2File(f) as ds:
                    total_z += ds.sizes.get('Z', 1)
            except Exception as e:
                print(f"Error scanning file {f}: {str(e)}")
    
    # Ensure total_z is defined before using it in the progress bar
    if total_z == 0:
        print("No Z-slices found. Exiting.")
        return
    
    # Main processing with progress bar
    start_time = time.time()
    with tqdm(total=total_z, desc="Converting", unit="slice", dynamic_ncols=True) as pbar:
        for file_path in nd2_files:
            convert_nd2_to_tif(file_path, output_directory, pbar)
        
        # Print elapsed time at the end
        elapsed_time = time.time() - start_time
        print(f"Conversion completed in {elapsed_time:.2f} seconds.")

In [5]:
# Main execution block
if __name__ == "__main__":
    # Set input and output directories
    data_dir = data_directory
    output_dir = data_dir
    
    batch_convert_nd2_to_tif(data_dir, output_dir)

Number of .nd2 files found: 48


Converting:  53%|████████████████▊               | 480/912 [2:17:47<2:04:00, 17.22s/slice, File=WellD12_Channel..., Elapsed=8266.9s]

Conversion completed in 8267.39 seconds.
